# PART 1

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
import skimage
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks
from matplotlib import cm
from skimage.segmentation import slic, mark_boundaries
from skimage.color import label2rgb
from scipy.io import loadmat,savemat
import scipy.io
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

## Main Code

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2


class FindRotationAngleWithPlots:
    def __init__(self, original_path, rotated_path, canny_thresholds, hough_params, bins, line_params):
        self.original_path = original_path
        self.rotated_path = rotated_path
        self.canny_thresholds = canny_thresholds
        self.hough_params = hough_params
        self.bins = bins
        self.line_params = line_params

    def read_and_preprocess_image(self, image_path):

        image = cv2.imread(image_path)
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return image, gray_img

    def detect_edges(self, gray_img):
        low, high = self.canny_thresholds
        low = int(low * 255)
        high = int(high * 255)
        edges = cv2.Canny(gray_img, low, high, apertureSize=3)
        return edges

    def hough_transform(self, edges):
        rho, theta, threshold = self.hough_params
        min_line_length, max_line_gap = self.line_params

        lines = cv2.HoughLinesP(edges, rho, theta, threshold,
                                minLineLength=min_line_length, maxLineGap=max_line_gap)
        return lines
    def print_edges(self,edges):
        
        for i, (edge) in enumerate(edges):
            plt.subplot(4, len(edges) // 4 + 1, i + 1)
            plt.imshow(edge, cmap='gray')
            plt.axis('off')

        
            
    def create_histogram(self, lines):
        
        bin_count = self.bins
        bin_vals = np.zeros(bin_count)
        bin_edges = np.linspace(0, 180, bin_count + 1)
        rotation_angles=[]
        if lines is not None:
            for line in lines:
                for x1, y1, x2, y2 in line:
                    theta = np.degrees(np.arctan2(y2 - y1, x2 - x1)) 
                    rotation_angles.append(theta)
                    theta=theta%180
                    
                    bin_idx = np.digitize([theta], bin_edges) - 1
                    line_length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
                    bin_vals[bin_idx[0]] += line_length
                    
        return bin_vals,rotation_angles

    def circular_rotate_right(self, array, k):
        return np.roll(array, k)

    def find_matches(self, original_histograms, rotated_histograms,rotations):
        bin_counts = self.bins
        matches = []

        for i, rotate_hist in enumerate(rotated_histograms):
            min_val = float('inf')
            best_match = None
            best_rotation_angle = None

            for j, orig_hist in enumerate(original_histograms):
                for k in range(bin_counts):
                    rotated_hist = self.circular_rotate_right(rotate_hist, k)
                    dist = np.sum((rotated_hist - orig_hist) ** 2)

                    if dist < min_val:
                        min_val = dist
                        best_match = j
                        best_rotation_angle = k * 180 / bin_counts
                        if rotations[i][0]>=0:
                            best_rotation_angle=180-best_rotation_angle
                        #best_rotation_angle = 180-best_rotation_angle if best_rotation_angle < 90 else best_rotation_angle
                        

            matches.append((i, best_match, best_rotation_angle))
        return matches

    def plot_edges(self, images, edges_list, titles):
        
        for i, (image, edges, title) in enumerate(zip(images, edges_list, titles)):
            fig, axes = plt.subplots(1, 2, figsize=(15, 7))
            axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f"Original: {title}")
            axes[0].axis('off')

            axes[1].imshow(edges, cmap='gray')
            axes[1].set_title(f"Edges: {title}")
            axes[1].axis('off')

            plt.tight_layout()
            plt.show()

    def plot_hough_lines(self, images, lines_list, titles):

        

        num_images = len(images)
        cols = 4  # Number of columns in the grid
        rows = (num_images // cols) + (num_images % cols > 0)  # Calculate rows based on the number of images
    
        plt.figure(figsize=(15, 10))
        for i, (image, lines, title) in enumerate(zip(images, lines_list, titles)):
            hough_image = image.copy()
            if lines is not None:
                for line in lines:
                    for x1, y1, x2, y2 in line:
                        cv2.line(hough_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            plt.subplot(rows, cols, i + 1)  # Add a subplot for each image
            plt.imshow(cv2.cvtColor(hough_image, cv2.COLOR_BGR2RGB))
            plt.title(f"Hough Lines: {title}")
            plt.axis("off")
    
        plt.tight_layout()
        

    def plot_histograms(self, histograms, titles):
        
        plt.figure(figsize=(15, 10))
        for i, (hist, title) in enumerate(zip(histograms, titles)):
            plt.subplot(4, len(histograms) // 4 + 1, i + 1)
            plt.bar(range(len(hist)), hist, alpha=0.7)
            plt.title(f"Histogram: {title}")
            plt.xlabel("Bins")
            plt.ylabel("Frequency")

        plt.tight_layout()
        

    def plot_histograms_t(self, histograms, titles,histograms1, titles1):
        
        
    
        plt.figure(figsize=(15, 10))
        for i, (hist, hist2) in enumerate(zip(histograms, histograms1)):
             
            plt.subplot(4, len(histograms) // 4 + 1, i + 1)
            plt.bar(range(len(hist)), hist, alpha=0.7,label="Original Histogram")
            plt.bar(range(len(hist2)), hist2, alpha=0.7,label="Rotated Histogram")
            plt.title(f"{titles[i]} vs {titles1[i]}")
            plt.legend()
            plt.xlabel("Bins")
            plt.ylabel("Lenghts")
        
        plt.tight_layout()
        output_path = f'Together_Histograms_h_t{self.hough_params[2]}_bin{self.bins}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
        plt.show()

    def process_images(self):

        original_images = [os.path.join(self.original_path, f) for f in os.listdir(self.original_path)
                           if f.endswith(('.jpg', '.png', '.jpeg'))]
        rotated_images = [os.path.join(self.rotated_path, f) for f in os.listdir(self.rotated_path)
                          if f.endswith(('.jpg', '.png', '.jpeg'))]

        original_images_data, rotated_images_data = [], []
        original_edges, rotated_edges = [], []
        original_lines, rotated_lines = [], []

        original_histograms, rotated_histograms = [], []
        rotations=[]

        for img_path in original_images:
            img, gray_img = self.read_and_preprocess_image(img_path)
            edges = self.detect_edges(gray_img)
            lines = self.hough_transform(edges)
            histogram,_ = self.create_histogram(lines)

            original_images_data.append(img)
            original_edges.append(edges)
            original_lines.append(lines)
            original_histograms.append(histogram)
        for img_path in rotated_images:
            img, gray_img = self.read_and_preprocess_image(img_path)
            edges = self.detect_edges(gray_img)
            lines = self.hough_transform(edges)
            histogram,rotation = self.create_histogram(lines)
            rotations.append(rotation)
            
            rotated_images_data.append(img)
            rotated_edges.append(edges)
            rotated_lines.append(lines)
            rotated_histograms.append(histogram)

        matches = self.find_matches(original_histograms, rotated_histograms,rotations)
        plt.figure(figsize=(15,10))
        plt.title(f"Canny Edges of Original images with parameters Low Threshold={int(self.canny_thresholds[0]*255)},High Threshold={int(self.canny_thresholds[1]*255)}")      
        self.print_edges(original_edges)
        plt.tight_layout()
        output_path = f'Canny_Original_{self.canny_thresholds[0]}_{self.canny_thresholds[1]}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
        plt.show()
        
        plt.figure(figsize=(15,10))
        plt.title(f"Canny Edges of Rotated images with parameters Low Threshold={int(self.canny_thresholds[0]*255)},High Threshold={int(self.canny_thresholds[1]*255)}")      
        self.print_edges(rotated_edges)
        plt.tight_layout()
        output_path = f'Canny_Rotated_{self.canny_thresholds[0]}_{self.canny_thresholds[1]}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
        plt.show()
        # Plot results
        self.plot_edges(original_images_data, original_edges, [os.path.basename(p) for p in original_images])
        self.plot_edges(rotated_images_data, rotated_edges, [os.path.basename(p) for p in rotated_images])

        self.plot_hough_lines(original_images_data, original_lines, [os.path.basename(p) for p in original_images])
        output_path =f'Original_hough_t{self.hough_params[2]}_p{self.line_params[0]}_{self.line_params[1]}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
            
        plt.show()
        self.plot_hough_lines(rotated_images_data, rotated_lines, [os.path.basename(p) for p in rotated_images])
        output_path =f'Rotated_hough_t{self.hough_params[2]}_p{self.line_params[0]}_{self.line_params[1]}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
        
        plt.show()

        self.plot_histograms(original_histograms, [os.path.basename(p) for p in original_images])
        output_path =f'Original_Histograms_h_t{self.hough_params[2]}_bin{self.bins}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
        
        plt.show()
        self.plot_histograms(rotated_histograms, [os.path.basename(p) for p in rotated_images])
        output_path = f'Rotated_Histograms_h_t{self.hough_params[2]}_bin{self.bins}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Use high DPI and remove extra margins
        
        plt.show()

        self.plot_histograms_t(original_histograms, [os.path.basename(p) for p in original_images],rotated_histograms, [os.path.basename(p) for p in rotated_images])
        
        for i, match in enumerate(matches):
            rotated_idx, original_idx, angle = match
            image1 = cv2.imread(f"{original_images[original_idx]}")
            image2 = cv2.imread(f"{rotated_images[rotated_idx]}")
            print(f"{rotated_images[rotated_idx]} & {original_images[original_idx]}& {angle:.2f}°")
            #print(f"Rotated Image {rotated_images[rotated_idx]} matched with Original Image {original_images[original_idx]}")
            #print(f"Rotation Angle: {angle:.2f}°")
            #fig, axes = plt.subplots(1, 2, figsize=(10,5))
            #axes[0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
            #axes[0].axis('off')
            #axes[1].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
            #axes[1].axis('off')
            #plt.tight_layout()
            #plt.show()
        return matches


# Configuration
original_path = "part1/img/"
rotated_path = "part1/rotated_img/"
canny_thresholds = (0.1, 0.5)  # Normalized thresholds
hough_params = (1, np.pi / 180, 6)  # Rho, Theta, Threshold
bins = 60  # Number of bins for the histogram
line_params = (8, 6)  # Min line length, Max line gap

# Run the updated process with plotting
rotation_estimator = FindRotationAngleWithPlots(original_path, rotated_path, canny_thresholds, hough_params, bins, line_params)
matches = rotation_estimator.process_images()